## Web Scraping - Sketch Engine

In [1]:
from bs4 import BeautifulSoup, NavigableString
# from urllib2 import urlopen
from urllib.request import urlopen
import os

import requests
from lxml import html

import json
import pandas as pd

import time
from time import gmtime, strftime

In [2]:
def getSketchEngineCollocations(lemma, posTag, colType):

    lstSketchEngineCols = []
    # URL to get the requests
    startUrl = 'https://the.sketchengine.co.uk/bonito/corpus/wsketch?lemma='
    lempos = '&lpos=-' 
    endUrl = '&minfreq=auto&minscore=0&maxitems=1000&clustercolls=0&minsim=0.15&sort_gramrels=1&sort_ws_columns=s&corpname=preloaded%2Fbawe2&format=json'

    requestUrl = startUrl + lemma + lempos + posTag + endUrl

    # Login credentials
    payload = { 'username' : 'api_testing', 'pass': 'YNSC0B9OXN57XB48T9HWUFFLPY4TZ6OE'}

    ### Login and set Context
    with requests.Session() as session:
    #     r = session.post('https://the.sketchengine.co.uk/bonito/corpus/wsketch?lemma=switch&lpos=-V&minfreq=auto&minscore=0&maxitems=12&clustercolls=0&minsim=0.15&sort_gramrels=1&sort_ws_columns=s&corpname=preloaded%2Fbnc2_tt21&format=json', data=payload)
    #     r = session.post('https://the.sketchengine.co.uk/bonito/corpus/wsketch?lemma=switch&lpos=-V&minfreq=auto&minscore=0&maxitems=12&clustercolls=0&minsim=0.15&sort_gramrels=1&sort_ws_columns=s&corpname=preloaded%2Fbawe2&format=json', data=payload)
        
#         r = session.post('https://the.sketchengine.co.uk/login', data=payload)
#         r = session.get(requestUrl)
        
        r = session.post(requestUrl, data=payload)
        respObj = r.json()

        for key, value in respObj.items():
            if key == 'Gramrels':
                for dictGramrels in respObj[key]:
                    if dictGramrels['name'] == colType:
#                         print('Headword, Headword_Lempos, Collocateword, Collocateword_Lempos, Collocateword_Filterwords, Count, Score')
                        for item in (dictGramrels['Words']):
                            lstSketchEngineCols.append((lemma, lemma + '-' + posTag, item['word'], item['lempos'], item['filterwords'], item['count'], item['score']))

    return lstSketchEngineCols

In [3]:
# Change path to the folder where all collocation data files will be stored
colFolder = r'D:\Education\vocabexpert\emnlp_paper\eval\collocations\sketchengine' 

# Change path to the folder where all reference files are stored. 
# Basically, this folder should then have 2 folders of 'se_flax' and 'elia' which contain the reference files.
%cd D:\Education\vocabexpert\acl_bea_paper\eval\reference
%pwd

D:\Education\vocabexpert\acl_bea_paper\eval\reference


'D:\\Education\\vocabexpert\\acl_bea_paper\\eval\\reference'

In [4]:
def getWordList(wordListCSVFile):
    colNameWord = 'Headword'
    colNamePOS = 'POS'
    df = pd.read_csv(wordListCSVFile)
    
    words = df[colNameWord].tolist()
    posTags = df[colNamePOS].tolist()
    
    wordList = list(zip(words, posTags))
    
    wordList = list(set(wordList))
    
    wordList = sorted(wordList)
    
    return wordList

In [5]:
lstRefColTypesFileNames = ['n1_n2', 'n2_n1', 'n2_v1', 'n2_adj1', 'v1_n2', 'v1_adj2', 'v1_adv2', 
                           'v2_adv1', 'adj1_n2', 'adj2_v1', 'adj2_adv1']
lstRefColTypesSE = ['modifies', 'modifier', 'object_of', 'modifier', 'object', 'adj_comp', 'modifier', 
                    'modifier', 'modifies', 'adj_comp_of', 'modifier']

In [6]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

for i in range(11): # Use time delay (sleep) as there is a limit on the number of hits.
#     i = 5

    colTypeFileName = lstRefColTypesFileNames[i]
    colType = lstRefColTypesSE[i]
    
    wordListCSVFile = './se_flax/ACL - Reference set - ' + colTypeFileName + '.csv'
    lstWords = getWordList(wordListCSVFile) #Read from word list
    print(colTypeFileName + ' : ' , str(len(lstWords)))
    
    colSketchEngine = os.path.join(colFolder, colTypeFileName)
    
    for word in lstWords:
        print(word)
        lemma = word[0].lower()
        posTag = word[1].capitalize()
        if posTag == 'Adj':
            posTag = 'J'
        
        time.sleep(100)
        lstSketchEngineCols = getSketchEngineCollocations(lemma, posTag, colType)

        if len(lstSketchEngineCols) != 0:
            # Store the file            
            final_directory = colSketchEngine
            if not os.path.exists(final_directory):
                os.makedirs(final_directory)
            fname = final_directory + '/' + lemma + '_' + posTag + '_' + colTypeFileName + '+' + colType

            fname += '.csv'
            
            headers = ['HeadWord', 'Headword_Lempos', 'CollocateWord', 'CollocateWord_Lempos',  'Collocateword_Filterwords', 'Count', 'Score']
            df = pd.DataFrame(lstSketchEngineCols, columns=headers)
            df.to_csv(fname, index=False)

        else:
            print('This word doesnt have any collocation type generated! - ' + str(i) + ' ' + lemma + '_' + posTag + '_' + colTypeFileName + '+' + colType)


print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2018-05-18 17:32:19
v1_adj2 :  8
('become', 'v')
('consider', 'v')
('deem', 'v')
('get', 'v')
('make', 'v')
('prove', 'v')
('remain', 'v')
('seem', 'v')
2018-05-18 17:45:49
